In [2]:
#Automated Wescraping Tool
#From Cibil to Zauba
  
import tkinter as tk
import tkinter.messagebox
import time
import glob
import os
import xlrd
import openpyxl
import requests as req


from tkinter import ttk 
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from openpyxl import Workbook, load_workbook
from datetime import datetime

now = datetime.now()

#Store file by time
date_time = now.strftime("%m%d_%H%M%S")
path = "D:/Results/MyData " + date_time + ".xlsx"
 
#Directors of company
def addDirectors(company,link,dirobj):
    if link==None:
        return None
    if link=="Company URL Not Found":
        c4=dirobj
        reply="Company URL Not Found"
        c4.value=reply
        return None
    else:    
        link=link.replace("company","company-directors")
        r=req.get(link)
        soup=BeautifulSoup(r.content)
        attr={"class":"table table-striped"}
        names=[]
        for i in soup.find_all("table",attr):
            for j in i.find_all("tr"):
                for k in j.find_all("td"):
                    for l in k.find_all("h5"):
                        for m in k.find_all("a"):
                            if m.text.isupper():
                                names.append(m.text)
        
        c4=dirobj
        if not names:
            c4.value="No Director"   #No values in names
        else:
            for i,e in enumerate(names):
                if c4.value=='' or c4.value==None:
                    c4.value=e
                else:
                    c4.value=c4.value+', '+e
        print(names)
        return None

    
#Returns CIN Number
def cinSearch(compname):
    if compname=="":
        return None
    
    else:
        company=compname.replace(" ","-")
        company=company.lower()
        
        url="https://www.zaubacorp.com/companysearchresults/"+company
        attr={"class":"container basic-cont"}
        r=req.get(url)
        soup=BeautifulSoup(r.content)

        l=soup.find_all("div",attr)
        child=list(l[0].children)
        text=str(child[2])
        
        if text[4:len(text)-4].startswith("We are unable to find"):
            return "Company CIN Not Found"
            
        else:
            l=soup.find_all("h5")
            id=str(l[0])[4:len(l[0])-6]
            return id
    
#Gets Company URL
def companySearch(compname):
    if compname=="":
        return None
    
    else:
        company=compname.replace(" ","-")
        company=company.lower()
        
        url="https://www.zaubacorp.com/companysearchresults/"+company
        attr={"class":"container basic-cont"}
        r=req.get(url)
        soup=BeautifulSoup(r.content)

        l=soup.find_all("div",attr)
        child=list(l[0].children)
        text=str(child[2])
        
        if text[4:len(text)-4].startswith("We are unable to find"):
            return "Company URL Not Found"
        
        else:
            l=soup.find_all("h5")
            
            id=str(l[0])[4:len(l[0])-6]
            
            url="https://www.zaubacorp.com/company/"+company.upper()+"/"+id           
            
            return url
        
    
def processFile():
    
    #Gets the location of the last modified folder       
    list_of_files = glob.glob('C:/Users/Raja/Downloads/*')
    latest_file = max(list_of_files, key=os.path.getctime)
    print (latest_file)

    #Opens downloaded excel file from Cibil
    locfile = latest_file.replace("\\","/")
    wb_old=xlrd.open_workbook(locfile)
    sheet_old=wb_old.sheet_by_index(0)

    #DataBase excel storage
    wb_new = openpyxl.Workbook() 
    sheet_new = wb_new.active

    #Column Heading for new excel
    sheet_new.cell(row= 1 , column = 1).value="LIST OF COMPANIES"
    sheet_new.cell(row= 1 , column = 2).value="Company URL"
    sheet_new.cell(row= 1 , column = 3).value="Company DIN Number"
    sheet_new.cell(row= 1 , column = 4).value="Director Name"

    #fills Column for new excel 
    for i in range(3,sheet_old.nrows-1):
        c1=sheet_new.cell(row= i-1 , column = 1)  
        c1.value= sheet_old.cell_value(i,1)         #Stores company names
    
        c2=sheet_new.cell(row= i-1 , column = 2)
        c2.value=companySearch(sheet_old.cell_value(i,1))   #stores UlRs of the Company 
    
        c3=sheet_new.cell(row= i-1 , column = 3)
        c3.value=cinSearch(sheet_old.cell_value(i,1))     #Stores CIN values
    
        #addDirectors(sheet.cell_value(i,1),c2.value,i)
        dirobj=sheet_new.cell(row= i-1 , column = 4)
        addDirectors(sheet_old.cell_value(i,1),c2.value,dirobj)  #function called to store Director names 
    
    wb_new.save(path)  #New ecel is saved

#downloads required excel from cibil
def downloadFile():
    myoptions = Options()
    #myoptions.add_argument("headless")
    myoptions.add_argument('window-size=1200x600')
    web = webdriver.Chrome(options=myoptions)
    
    # navigate to the page and set values
    web.get('https://suit.cibil.com/')
    WebDriverWait(web, 10)
    Select(web.find_element_by_xpath('//*[@id="quarterIdGrantors"]')).select_by_visible_text(monthchoosen.get())
    
    #click
    web.find_element_by_xpath('//*[@id="loadSuitFiledDataSearchAction"]/div[1]/div[2]/div[3]/img').click()

    #download 
    web.find_element_by_xpath('//*[@id="downloadReport"]/div').click()
    time.sleep(3)
    
    #display message
    tkinter.messagebox.showinfo( "File Download", "Downloaded file for " + monthchoosen.get())
    web.close()
    processFile()
    
    tkinter.messagebox.showinfo( "Finished", "Processed and saved in " + path)
     
# Creating tkinter window 
window = tk.Tk() 
window.title('Hackaton - 2020') 
window.geometry('500x250') 
  
# label text for title 
ttk.Label(window, text = "CKR - Web Scraping Tool",  
          background = 'green', foreground ="white",  
          font = ("Times New Roman", 15)).grid(row = 0, column = 1) 
  
# label 
ttk.Label(window, text = "Select the Date :", 
          font = ("Times New Roman", 10)).grid(column = 0, 
          row = 5, padx = 10, pady = 25) 

# Get Quater Date using Web Driver
myoptions = Options()
myoptions.add_argument("headless")  #Hide webdriver 
myoptions.add_argument('window-size=1200x600')
web = webdriver.Chrome(options=myoptions)

# navigate to the page(credit 1st)
web.get('https://suit.cibil.com/')
WebDriverWait(web, 2)

# Pickup Dates fom Cibil
DateValsxpath = web.find_element_by_xpath('//*[@id="quarterIdGrantors"]')
DateVals = DateValsxpath.text
web.close()
  
# Combobox creation 
n = tk.StringVar() 
monthchoosen = ttk.Combobox(window, width = 27, textvariable = n) 

DateVals = DateVals.replace('DATE\n','')
DateVals = DateVals.replace('\n',',')
monthchoosen['values'] = DateVals.split(",")
 
monthchoosen.grid(column = 1, row = 5) 
monthchoosen.current(0)  

msg = tkinter.Button(window, text ="Download File", command = downloadFile)
msg.grid(row=7,column=1)

window.mainloop() 